# Imports and Set-up

In [1]:
import sys
import pandas as pd

In [24]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import CSV_DATA_DIR

from src.biomass_equations import allometric_tropical_tree, vmd0001_eq1

In [44]:
# Filepaths
trees_reference = CSV_DATA_DIR / "arr_growth" / "tree-species.csv"
output_fpath = CSV_DATA_DIR / "arr_growth" / "yoy-sequestration.csv"

In [9]:
# variables
years = 25

## Read Data

In [6]:
tree_lookup = pd.read_csv(trees_reference)

In [7]:
tree_lookup

,species,common_name,scientific_name,eco_class,max_dbh_cm,max_height_m,wood_density_g_cm3,maid_cm_yr,maih_m_yr
0,dipterocarpaceae,apitong,Dipterocarpus grandiflorus,indigenous,135,50,0.67,0.69,0.71
1,dipterocarpaceae,yakal-saplungan,Hopea plagata,indigenous,120,45,0.89,0.57,0.75
2,dipterocarpaceae,bagtikan,Parashorea malaanonan,indigenous,200,60,0.44,0.62,0.89
3,dipterocarpaceae,almon,Shorea almon,indigenous,160,70,0.39,0.74,0.86
4,dipterocarpaceae,white lauan,Shorea contorta,endemic,182,50,0.59,0.87,0.97


# Project annual size and sequestration of trees

In [14]:
# Create empty lists to store the results
scientific_name = []
years_list = []
dbh_growth = []
height_growth = []

# Loop through each tree species to calculate the yearly growth
for _, row in tree_lookup.iterrows():
    for t in range(years + 1):  # From year 0 to 25
        scientific_name.append(row["scientific_name"])
        years_list.append(t)
        dbh_growth.append(t * row["maid_cm_yr"])  # Start from 0, and increase with MAI
        height_growth.append(
            t * row["maih_m_yr"]
        )  # Start from 0, and increase with MAI

# Create a new dataframe with the results
growth_df = pd.DataFrame(
    {
        "scientific_name": scientific_name,
        "year": years_list,
        "dbh_cm": dbh_growth,
        "height_m": height_growth,
    }
)

In [15]:
growth_df

,scientific_name,year,dbh_cm,height_m
0,Dipterocarpus grandiflorus,0,0.00,0.00
1,Dipterocarpus grandiflorus,1,0.69,0.71
2,Dipterocarpus grandiflorus,2,1.38,1.42
3,Dipterocarpus grandiflorus,3,2.07,2.13
4,Dipterocarpus grandiflorus,4,2.76,2.84
...,...,...,...,...
125,Shorea contorta,21,18.27,20.37
126,Shorea contorta,22,19.14,21.34
127,Shorea contorta,23,20.01,22.31
128,Shorea contorta,24,20.88,23.28


In [17]:
growth_df = growth_df.merge(tree_lookup[["scientific_name", "wood_density_g_cm3"]])

# Calculate biomass

In [21]:
growth_df = allometric_tropical_tree(
    growth_df, "wood_density_g_cm3", "dbh_cm", "height_m"
)

In [23]:
growth_df[growth_df["scientific_name"] == "Shorea contorta"]

,scientific_name,year,dbh_cm,height_m,wood_density_g_cm3,aboveground_biomass
104,Shorea contorta,0,0.00,0.00,0.59,0.000000
105,Shorea contorta,1,0.87,0.97,0.59,0.029744
106,Shorea contorta,2,1.74,1.94,0.59,0.226367
107,Shorea contorta,3,2.61,2.91,0.59,0.742008
108,Shorea contorta,4,3.48,3.88,0.59,1.722777
109,Shorea contorta,5,4.35,4.85,0.59,3.311171
110,Shorea contorta,6,5.22,5.82,0.59,5.647084
111,Shorea contorta,7,6.09,6.79,0.59,8.868384
112,Shorea contorta,8,6.96,7.76,0.59,13.111274
113,Shorea contorta,9,7.83,8.73,0.59,18.510557


# Calculate carbon stock of trees per year

In [27]:
# convert aboveground biomass to tonnes
growth_df["aboveground_biomass"] = growth_df["aboveground_biomass"] / 1000

In [29]:
growth_df = vmd0001_eq1(growth_df)

In [30]:
growth_df

,scientific_name,year,dbh_cm,height_m,wood_density_g_cm3,aboveground_biomass,aboveground_carbon_tonnes
0,Dipterocarpus grandiflorus,0,0.00,0.00,0.67,0.000000,0.000000
1,Dipterocarpus grandiflorus,1,0.69,0.71,0.67,0.000016,0.000007
2,Dipterocarpus grandiflorus,2,1.38,1.42,0.67,0.000120,0.000056
3,Dipterocarpus grandiflorus,3,2.07,2.13,0.67,0.000394,0.000185
4,Dipterocarpus grandiflorus,4,2.76,2.84,0.67,0.000915,0.000430
...,...,...,...,...,...,...,...
125,Shorea contorta,21,18.27,20.37,0.59,0.221236,0.103981
126,Shorea contorta,22,19.14,21.34,0.59,0.253519,0.119154
127,Shorea contorta,23,20.01,22.31,0.59,0.288760,0.135717
128,Shorea contorta,24,20.88,23.28,0.59,0.327081,0.153728


In [31]:
growth_df["annual_change_carbon_stock"] = growth_df.groupby("scientific_name")[
    "aboveground_carbon_tonnes"
].diff()

In [39]:
growth_df[growth_df["scientific_name"] == "Shorea contorta"]

,scientific_name,year,dbh_cm,height_m,wood_density_g_cm3,aboveground_biomass,aboveground_carbon_tonnes,annual_change_carbon_stock
104,Shorea contorta,0,0.00,0.00,0.59,0.000000,0.000000,NaN
105,Shorea contorta,1,0.87,0.97,0.59,0.000030,0.000014,0.000014
106,Shorea contorta,2,1.74,1.94,0.59,0.000226,0.000106,0.000092
107,Shorea contorta,3,2.61,2.91,0.59,0.000742,0.000349,0.000242
108,Shorea contorta,4,3.48,3.88,0.59,0.001723,0.000810,0.000461
109,Shorea contorta,5,4.35,4.85,0.59,0.003311,0.001556,0.000747
110,Shorea contorta,6,5.22,5.82,0.59,0.005647,0.002654,0.001098
111,Shorea contorta,7,6.09,6.79,0.59,0.008868,0.004168,0.001514
112,Shorea contorta,8,6.96,7.76,0.59,0.013111,0.006162,0.001994
113,Shorea contorta,9,7.83,8.73,0.59,0.018511,0.008700,0.002538


In [42]:
pivot_df = growth_df.pivot_table(
    index="year",  # Each row will correspond to a year
    columns="scientific_name",  # Each column will correspond to a tree species
    values="annual_change_carbon_stock",  # The annual change in carbon stock
    aggfunc="sum",  # In case there are multiple rows for the same year and species, sum the values
)

In [43]:
pivot_df

scientific_name,Dipterocarpus grandiflorus,Hopea plagata,Parashorea malaanonan,Shorea almon,Shorea contorta
year,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000007,0.000007,0.000005,0.000006,0.000014
2,0.000049,0.000047,0.000033,0.000040,0.000092
3,0.000129,0.000123,0.000086,0.000105,0.000242
4,0.000245,0.000235,0.000164,0.000200,0.000461
5,0.000396,0.000380,0.000266,0.000323,0.000747
6,0.000583,0.000559,0.000391,0.000475,0.001098
7,0.000804,0.000771,0.000540,0.000655,0.001514
8,0.001059,0.001015,0.000711,0.000863,0.001994


In [ ]:
pivot_df.to_csv(output_fpath)